En este scrip de python es donde uniremos los csv de 2017 y 2020 con el calculado aqui que es 2023 obteniendo el csv_Ense_all


In [1]:
from pathlib import Path
import pandas as pd

# Ruta base
base = Path(r"c:/Users/nuez_/Desktop/Proyecto ENSE")
raw_2023 = base / "Data_raw" / "2023"

list(raw_2023.iterdir())


[WindowsPath('c:/Users/nuez_/Desktop/Proyecto ENSE/Data_raw/2023/dr_ESdEadulto_2023.xlsx'),
 WindowsPath('c:/Users/nuez_/Desktop/Proyecto ENSE/Data_raw/2023/ESdEadulto_2023.tab')]

In [2]:
import pandas as pd

dic23_path = raw_2023 / "dr_ESdEadulto_2023.xlsx"

xls = pd.ExcelFile(dic23_path)
xls.sheet_names


['Diseño', 'Tablas1', 'Tablas2', 'Tablas3', 'Tablas4']

In [3]:
from pathlib import Path
import pandas as pd

# Carpeta base del proyecto
base = Path(r"c:/Users/nuez_/Desktop/Proyecto ENSE")

# Carpeta de datos brutos 2023
raw_2023 = base / "Data_raw" / "2023"

# Rutas de los ficheros 2023
dic23_path = raw_2023 / "dr_ESdEadulto_2023.xlsx"
txt23_path = raw_2023 / "ESdEadulto_2023.tab"    # ajusta el nombre si es distinto



In [4]:
# Cargar hoja correcta (la hoja se llama "Diseño")
tmp23 = pd.read_excel(dic23_path, sheet_name="Diseño", header=None)

# OJO: en el Excel 2023 el encabezado es "Variable", no "VARIABLE INE"
fila_header23 = tmp23[tmp23.eq("Variable").any(axis=1)].index[0]

# Crear diccionario limpio
dic23 = tmp23.iloc[fila_header23:]
dic23.columns = dic23.iloc[0]
dic23 = dic23.iloc[1:].reset_index(drop=True)

# Nos quedamos con las columnas relevantes
dic23 = dic23[["Variable", "Longitud", "Posición", "Descripción"]]

dic23.head()


1,Variable,Longitud,Posición,Descripción
0,CCAA,2,1,Comunidad Autónoma de residencia
1,IDENTHOGAR,7,3,Número de identificación del hogar: 0000001 a ...
2,NORDENa,2,10,Número de orden del adulto seleccionado
3,SEXOa,1,12,Identificación del adulto seleccionado: Sexo\n...
4,EDADa,3,13,Identificación del adulto seleccionado: Edad. ...


In [5]:
from pathlib import Path
import pandas as pd

# Ruta base (ajusta si la tuya es distinta)
base = Path(r"c:/Users/nuez_/Desktop/Proyecto ENSE")
raw_2023 = base / "Data_raw" / "2023"

# 1. Volvemos a tener el diccionario (por si no está en memoria)
dic23_path = raw_2023 / "dr_ESdEadulto_2023.xlsx"
tmp23 = pd.read_excel(dic23_path, sheet_name="Diseño", header=None)
fila_header23 = tmp23[tmp23.eq("Variable").any(axis=1)].index[0]
dic23 = tmp23.iloc[fila_header23:]
dic23.columns = dic23.iloc[0]
dic23 = dic23.iloc[1:].reset_index(drop=True)
dic23 = dic23[["Variable", "Longitud", "Posición", "Descripción"]]

# 2. Nos quedamos solo con filas con posición y longitud válidas
dic23 = dic23.dropna(subset=["Posición", "Longitud"]).copy()

dic23["Posición"] = dic23["Posición"].astype(int)
dic23["Longitud"] = dic23["Longitud"].astype(int)

# 3. Construimos colspecs: (inicio, fin) en base 0
colspecs23 = list(
    zip(dic23["Posición"] - 1, dic23["Posición"] + dic23["Longitud"] - 1)
)

nombres23 = dic23["Variable"].tolist()

# 4. Leemos el .tab de microdatos
txt23_path = raw_2023 / "ESdEadulto_2023.tab"

df23_raw = pd.read_fwf(
    txt23_path,
    colspecs=colspecs23,
    names=nombres23,
    dtype=str
)

df23_raw.head()


,CCAA,IDENTHOGAR,NORDENa,SEXOa,EDADa,PROXY_0,PROXY_1,PROXY_2,PROXY_2b,PROXY_4,...,T3,FACTORADULTO,CLASE_PR,IMC,CMD1,CMD2,CMD3,INDICE_BIENESTAR,SEVERIDAD_DEPRESIVA,CUADROS_DEPRESIVOS
0,"""C","CAA""\t""I",DE,N,THO,G,A,R,"""","""",...,5,"a_9""\t""C5b",_,9,"""\t""C5c","_9""\t""C","5a_10""","""C5b_",1,0
1,16,000000,1,0,1\t1,NaN,7,5,1,NaN,...,NaN,3\t7\t4\t0,2,NaN,2\t0\t0,9\t0\t1,3\t4\t4,4\t4\t1,2,NaN
2,16,000000,2,0,1\t2,NaN,9,6,2,NaN,...,NaN,1\t1\t1\t1\t1,NaN,5,2\t\t1,4\t1\t5,2\t2,2\t3\t2,3,NaN
3,16,000000,3,0,1\t2,NaN,7,4,1,NaN,...,2,3\t159\t73,NaN,1,2\t7\t4,0\t\t0,0\t9,0\t1\t2,3,NaN
4,16,000000,4,0,2\t2,NaN,7,9,1,NaN,...,2,3\t162\t60,NaN,1,3\t7\t3,0\t\t7,7\t0\t0,8\t0\t1,2,NaN


In [6]:
import pandas as pd

# Ruta al archivo .tab
txt23_path = raw_2023 / "ESdEadulto_2023.tab"

# Cargar microdatos (2023 ya NO es fixed width)
df23_raw = pd.read_csv(txt23_path, sep="\t", dtype=str)

df23_raw.head()


,CCAA,IDENTHOGAR,NORDENa,SEXOa,EDADa,PROXY_0,PROXY_1,PROXY_2,PROXY_2b,PROXY_4,...,T3,FACTORADULTO,CLASE_PR,IMC,CMD1,CMD2,CMD3,INDICE_BIENESTAR,SEVERIDAD_DEPRESIVA,CUADROS_DEPRESIVOS
0,16,0000001,01,1,75,1,NaN,NaN,NaN,NaN,...,NaN,547.893,5,3,0,0,0,84,1,3
1,16,0000002,01,2,96,2,2,2,NaN,1,...,NaN,714.313,3,2,0,0,0,24,4,1
2,16,0000003,01,2,74,1,NaN,NaN,NaN,NaN,...,NaN,567.742,6,3,0,0,0,80,1,3
3,16,0000004,02,2,79,1,NaN,NaN,NaN,NaN,...,NaN,1433.792,4,2,0,0,0,100,1,3
4,16,0000005,01,1,70,1,NaN,NaN,NaN,NaN,...,NaN,1543.118,5,2,4.29,0,10,100,1,3


In [7]:
df23_raw.columns = df23_raw.columns.str.strip()


In [8]:
keep_vars = [
    "CCAA", "SEXOa", "EDADa", "IMC", "freq_dulces", "freq_refrescos",
    "freq_fastfood", "freq_snacks", "ICU"
]

df23 = df23_raw[keep_vars].copy()


KeyError: "['freq_dulces', 'freq_refrescos', 'freq_fastfood', 'freq_snacks', 'ICU'] not in index"

In [ ]:
# Dulces
dic23[dic23["Descripción"].str.contains("dulces", case=False, na=False)]

# Refrescos
dic23[dic23["Descripción"].str.contains("refrescos", case=False, na=False)]

# Comida rápida
dic23[dic23["Descripción"].str.contains("comida rápida", case=False, na=False)]

# Aperitivos / snacks
dic23[dic23["Descripción"].str.contains("aperitivos|snacks|saladas", 
                                        case=False, na=False)]

# Índice de consumo de ultraprocesados (si existe)
dic23[dic23["Descripción"].str.contains("ultraproces", case=False, na=False)]


1,Variable,Longitud,Posición,Descripción


In [ ]:
# 1) DULCES
dulces23 = dic23[
    dic23["Descripción"].str.contains("dulces|bollería|dulce|galletas", case=False, na=False)
][["Variable", "Descripción"]]
dulces23


1,Variable,Descripción
350,P1_11,"Frecuencia de consumo de dulces (galletas, bol..."


In [ ]:
# 2) REFRESCOS
refrescos23 = dic23[
    dic23["Descripción"].str.contains("refrescos|bebidas azucaradas", case=False, na=False)
][["Variable", "Descripción"]]
refrescos23


1,Variable,Descripción
351,P1_12,Frecuencia de consumo de refrescos con azúcar


In [ ]:
# 4) APERITIVOS / SNACKS
snacks23 = dic23[
    dic23["Descripción"].str.contains("aperitivos|snacks|patatas fritas|ganchitos", case=False, na=False)
][["Variable", "Descripción"]]
snacks23


1,Variable,Descripción
353,P1_14,Frecuencia de consumo de aperitivos o comidas ...


In [ ]:
# 5) ÍNDICE / ULTRAPROCESADOS (si existiera algo así)
icu23 = dic23[
    dic23["Descripción"].str.contains("ultraproces|procesados", case=False, na=False)
][["Variable", "Descripción"]]
icu23


1,Variable,Descripción


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

base = Path(r"c:/Users/nuez_/Desktop/Proyecto ENSE")
raw_2023 = base / "Data_raw" / "2023"

txt23_path = raw_2023 / "ESdEadulto_2023.tab"   # ajusta el nombre si fuera distinto

df23_raw = pd.read_csv(txt23_path, 
                       sep="\t", 
                       dtype=str)   # todo como texto de momento

df23_raw.head()


,CCAA,IDENTHOGAR,NORDENa,SEXOa,EDADa,PROXY_0,PROXY_1,PROXY_2,PROXY_2b,PROXY_4,...,T3,FACTORADULTO,CLASE_PR,IMC,CMD1,CMD2,CMD3,INDICE_BIENESTAR,SEVERIDAD_DEPRESIVA,CUADROS_DEPRESIVOS
0,16,0000001,01,1,75,1,NaN,NaN,NaN,NaN,...,NaN,547.893,5,3,0,0,0,84,1,3
1,16,0000002,01,2,96,2,2,2,NaN,1,...,NaN,714.313,3,2,0,0,0,24,4,1
2,16,0000003,01,2,74,1,NaN,NaN,NaN,NaN,...,NaN,567.742,6,3,0,0,0,80,1,3
3,16,0000004,02,2,79,1,NaN,NaN,NaN,NaN,...,NaN,1433.792,4,2,0,0,0,100,1,3
4,16,0000005,01,1,70,1,NaN,NaN,NaN,NaN,...,NaN,1543.118,5,2,4.29,0,10,100,1,3


In [ ]:
# Variables necesarias para ENSE 2023
keep_vars_23 = [
    "CCAA",
    "SEXOa",
    "EDADa",
    "N1",        # altura
    "N2",        # peso
    "P1_11",     # dulces
    "P1_12",     # refrescos
    "P1_13",     # fast food
    "P1_14"      # snacks
]

# Verificamos qué variables existen realmente en el TXT
print("Variables presentes:", [v for v in keep_vars_23 if v in df23_raw.columns])
print("Variables ausentes:", [v for v in keep_vars_23 if v not in df23_raw.columns])

# Crear df solo con las variables presentes
df23 = df23_raw[[v for v in keep_vars_23 if v in df23_raw.columns]].copy()


Variables presentes: ['CCAA', 'SEXOa', 'EDADa', 'N1', 'N2', 'P1_11', 'P1_12', 'P1_13', 'P1_14']
Variables ausentes: []


In [ ]:
df23 = df23.rename(columns={
    "SEXOa": "sexo",
    "EDADa": "edad",
    "N1": "altura_cm",
    "N2": "peso_kg",
    "P1_11": "freq_dulces",
    "P1_12": "freq_refrescos",
    "P1_13": "freq_fastfood",
    "P1_14": "freq_snacks"
})


In [ ]:
df23["año"] = 2023


In [ ]:
df23

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,año
0,16,1,75,162,68,5,6,6,5,2023
1,16,2,96,150,52,1,6,6,6,2023
2,16,2,74,159,73,1,6,6,5,2023
3,16,2,79,162,60,1,6,6,5,2023
4,16,1,70,175,70,1,6,5,5,2023
...,...,...,...,...,...,...,...,...,...,...
21027,19,2,33,165,69,4,4,4,4,2023
21028,19,2,31,170,65,4,4,4,4,2023
21029,19,1,39,170,65,4,4,4,4,2023
21030,19,1,42,180,74,4,4,4,4,2023


In [ ]:
import numpy as np
import pandas as pd


def calcular_icu(df):
    # Aseguramos que las 4 frecuencias son numéricas
    for col in ["freq_dulces", "freq_refrescos", "freq_fastfood", "freq_snacks"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        # Quitamos códigos 8/9 (NS/NC, etc.)
        df.loc[df[col].isin([8, 9]), col] = np.nan

    # Mapa de categorías → veces por semana
    map_freq_sem = {
        1: 7,     # Diario
        2: 5,     # 4–6 veces/semana
        3: 3,     # 2–3 veces/semana
        4: 1.5,   # 1 vez/semana
        5: 0.5,   # 1–3/mes
        6: 0,     # Nunca
    }

    for col in ["freq_dulces", "freq_refrescos", "freq_fastfood", "freq_snacks"]:
        df[col + "_sem"] = df[col].map(map_freq_sem)

    # Índice de consumo de ultraprocesados
    df["ICU"] = (
        df["freq_dulces_sem"] +
        df["freq_refrescos_sem"] +
        df["freq_fastfood_sem"] +
        df["freq_snacks_sem"]
    )

    # Misma clasificación para TODOS los años (la de 2020)
    df["ICU_categoria"] = pd.cut(
        df["ICU"],
        bins=[-0.1, 3, 7, 100],
        labels=["Bajo", "Moderado", "Alto"]
    )

    return df


In [ ]:
df23 = calcular_icu(df23)

In [ ]:
# Tomamos los nombres de las variables directamente del diccionario
var_dulces23     = dulces23["Variable"].iloc[0]       # P1_11
var_refrescos23  = refrescos23["Variable"].iloc[0]    # P1_12
var_snacks23     = snacks23["Variable"].iloc[0]       # P1_14

# Buscamos comida rápida en el diccionario
comida23 = dic23[dic23["Descripción"]
                 .str.contains("comida rápida", case=False, na=False)][["Variable","Descripción"]]
comida23

var_fastfood23   = comida23["Variable"].iloc[0]       # debería ser P1_13
print(var_dulces23, var_refrescos23, var_fastfood23, var_snacks23)


P1_11 P1_12 P1_13 P1_14


In [ ]:
df23

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,año,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria
0,16,1,75,162,68,5.0,6.0,6.0,5.0,2023,0.5,0.0,0.0,0.5,1.0,Bajo
1,16,2,96,150,52,1.0,6.0,6.0,6.0,2023,7.0,0.0,0.0,0.0,7.0,Moderado
2,16,2,74,159,73,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto
3,16,2,79,162,60,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto
4,16,1,70,175,70,1.0,6.0,5.0,5.0,2023,7.0,0.0,0.5,0.5,8.0,Alto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21027,19,2,33,165,69,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado
21028,19,2,31,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado
21029,19,1,39,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado
21030,19,1,42,180,74,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado


In [ ]:
# Creamos un df más limpio solo con lo que necesitamos
df23 = pd.DataFrame()

# CCAA y variables básicas
df23["CCAA"] = df23_raw["CCAA"]

# Sexo
df23["sexo"] = df23_raw["SEXOa"].astype(float).map({1: "Hombre", 2: "Mujer"})

# Edad
df23["edad"] = pd.to_numeric(df23_raw["EDADa"], errors="coerce")

# IMC numérico (si lo necesitas luego)
df23["IMC"] = pd.to_numeric(df23_raw["IMC"], errors="coerce")

# Frecuencias crudas de consumo (valores 1–6)
df23["freq_dulces"]    = pd.to_numeric(df23_raw[var_dulces23],    errors="coerce")
df23["freq_refrescos"] = pd.to_numeric(df23_raw[var_refrescos23], errors="coerce")
df23["freq_fastfood"]  = pd.to_numeric(df23_raw[var_fastfood23],  errors="coerce")
df23["freq_snacks"]    = pd.to_numeric(df23_raw[var_snacks23],    errors="coerce")

df23.head()


,CCAA,sexo,edad,IMC,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks
0,16,Hombre,75,3,5,6,6,5
1,16,Mujer,96,2,1,6,6,6
2,16,Mujer,74,3,1,6,6,5
3,16,Mujer,79,2,1,6,6,5
4,16,Hombre,70,2,1,6,5,5


In [ ]:
# Altura
dic23[dic23["Descripción"].str.contains("altura", case=False, na=False)][["Variable", "Descripción"]]


1,Variable,Descripción
324,N1,Altura en cm (50 a 220)


In [ ]:
# Peso
dic23[dic23["Descripción"].str.contains("peso", case=False, na=False)][["Variable", "Descripción"]]


1,Variable,Descripción
325,N2,Peso en kg (25 a 180)


In [ ]:
dic23[dic23["Descripción"].str.contains(
    "alcohol|bebidas alcohólicas|vino|cerveza|licor",
    case=False,
    na=False
)][["Variable", "Descripción"]]


In [ ]:
df23_raw["CMD1"].head(20)
df23_raw["CMD1"].value_counts(dropna=False).head(20)


In [ ]:
df23["alcohol_md_sem"] = df23_raw["CMD1"]
df23

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,año,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria,alcohol_md_sem
0,16,1,75,162,68,5.0,6.0,6.0,5.0,2023,0.5,0.0,0.0,0.5,1.0,Bajo,0
1,16,2,96,150,52,1.0,6.0,6.0,6.0,2023,7.0,0.0,0.0,0.0,7.0,Moderado,0
2,16,2,74,159,73,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto,0
3,16,2,79,162,60,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto,0
4,16,1,70,175,70,1.0,6.0,5.0,5.0,2023,7.0,0.0,0.5,0.5,8.0,Alto,4.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21027,19,2,33,165,69,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,0
21028,19,2,31,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,2.86
21029,19,1,39,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,8.57
21030,19,1,42,180,74,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,5.71


In [ ]:
# Aseguramos numéricos
df23["peso_kg"] = pd.to_numeric(df23["peso_kg"], errors="coerce")
df23["altura_cm"] = pd.to_numeric(df23["altura_cm"], errors="coerce")

# IMC
df23["IMC"] = df23["peso_kg"] / ( (df23["altura_cm"] / 100) ** 2 )


In [ ]:
df23["IMC_cat"] = pd.cut(
    df23["IMC"],
    bins=[0, 18.5, 25, 30, 100],
    labels=["Bajo peso", "Normopeso", "Sobrepeso", "Obesidad"]
)


In [ ]:
df23

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,año,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria,alcohol_md_sem,IMC,IMC_cat
0,16,1,75,162,68,5.0,6.0,6.0,5.0,2023,0.5,0.0,0.0,0.5,1.0,Bajo,0,25.910684,Sobrepeso
1,16,2,96,150,52,1.0,6.0,6.0,6.0,2023,7.0,0.0,0.0,0.0,7.0,Moderado,0,23.111111,Normopeso
2,16,2,74,159,73,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto,0,28.875440,Sobrepeso
3,16,2,79,162,60,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto,0,22.862369,Normopeso
4,16,1,70,175,70,1.0,6.0,5.0,5.0,2023,7.0,0.0,0.5,0.5,8.0,Alto,4.29,22.857143,Normopeso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21027,19,2,33,165,69,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,0,25.344353,Sobrepeso
21028,19,2,31,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,2.86,22.491349,Normopeso
21029,19,1,39,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,8.57,22.491349,Normopeso
21030,19,1,42,180,74,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,5.71,22.839506,Normopeso


In [ ]:
import pandas as pd

# CCAA: quedarnos solo con dígitos, convertir a número
df23["CCAA"] = (df23["CCAA"]
                .astype(str)
                .str.extract(r"(\d+)", expand=False))
df23["CCAA"] = pd.to_numeric(df23["CCAA"], errors="coerce").astype("Int64")

# sexo: igual
df23["sexo"] = (df23["sexo"]
                .astype(str)
                .str.extract(r"(\d+)", expand=False))
df23["sexo"] = pd.to_numeric(df23["sexo"], errors="coerce").astype("Int64")


In [ ]:
map_sexo = {1: "Hombre", 2: "Mujer"}

map_ccaa = {
    1: "Andalucía", 2: "Aragón", 3: "Asturias", 4: "Islas Baleares",
    5: "Canarias", 6: "Cantabria", 7: "Castilla y León", 8: "Castilla-La Mancha",
    9: "Cataluña", 10: "Comunidad Valenciana", 11: "Extremadura", 12: "Galicia",
    13: "Madrid", 14: "Murcia", 15: "Navarra", 16: "País Vasco",
    17: "La Rioja", 18: "Ceuta", 19: "Melilla"
}

df23["sexo"] = df23["sexo"].map(map_sexo)
df23["CCAA"] = df23["CCAA"].map(map_ccaa)


In [ ]:
df23

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,año,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria,alcohol_md_sem,IMC,IMC_cat
0,NaN,NaN,75,162,68,5.0,6.0,6.0,5.0,2023,0.5,0.0,0.0,0.5,1.0,Bajo,0,25.910684,Sobrepeso
1,NaN,NaN,96,150,52,1.0,6.0,6.0,6.0,2023,7.0,0.0,0.0,0.0,7.0,Moderado,0,23.111111,Normopeso
2,NaN,NaN,74,159,73,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto,0,28.875440,Sobrepeso
3,NaN,NaN,79,162,60,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto,0,22.862369,Normopeso
4,NaN,NaN,70,175,70,1.0,6.0,5.0,5.0,2023,7.0,0.0,0.5,0.5,8.0,Alto,4.29,22.857143,Normopeso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21027,NaN,NaN,33,165,69,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,0,25.344353,Sobrepeso
21028,NaN,NaN,31,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,2.86,22.491349,Normopeso
21029,NaN,NaN,39,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,8.57,22.491349,Normopeso
21030,NaN,NaN,42,180,74,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,5.71,22.839506,Normopeso


In [ ]:
df23["CCAA"].dtype, df23["sexo"].dtype
df23["CCAA"].unique()[:20]
df23["sexo"].unique()[:10]


array([nan], dtype=object)

In [ ]:

df23_raw[["CCAA"]].head()


,CCAA
0,16
1,16
2,16
3,16
4,16


In [ ]:
df23_raw[["SEXOa"]].head()

,SEXOa
0,1
1,2
2,2
3,2
4,1


In [ ]:
df23.columns


Index(['CCAA', 'sexo', 'edad', 'altura_cm', 'peso_kg', 'freq_dulces',
       'freq_refrescos', 'freq_fastfood', 'freq_snacks', 'año',
       'freq_dulces_sem', 'freq_refrescos_sem', 'freq_fastfood_sem',
       'freq_snacks_sem', 'ICU', 'ICU_categoria', 'alcohol_md_sem', 'IMC',
       'IMC_cat'],
      dtype='object')

In [ ]:
# Recuperar columnas base desde el raw
df23["CCAA"] = df23_raw["CCAA"]
df23["sexo"] = df23_raw["SEXOa"]


In [ ]:
df23[["CCAA", "sexo"]].head()


,CCAA,sexo
0,16,1
1,16,2
2,16,2
3,16,2
4,16,1


In [ ]:
import numpy as np
import pandas as pd

# 1) RECONSTRUIR desde df23_raw (para no arrastrar NaNs)
df23["CCAA"] = pd.to_numeric(df23_raw["CCAA"], errors="coerce").astype("Int64")
df23["sexo"] = pd.to_numeric(df23_raw["SEXOa"], errors="coerce").astype("Int64")

# 2) Mapas
map_ccaa = {
    1:"Andalucía",2:"Aragón",3:"Asturias",4:"Illes Balears",5:"Canarias",6:"Cantabria",
    7:"Castilla y León",8:"Castilla-La Mancha",9:"Cataluña",10:"C. Valenciana",11:"Extremadura",
    12:"Galicia",13:"Madrid",14:"Región de Murcia",15:"Navarra",16:"País Vasco",
    17:"La Rioja",18:"Ceuta",19:"Melilla"
}
map_sexo = {1: "Hombre", 2: "Mujer"}

df23["CCAA"] = df23["CCAA"].map(map_ccaa)
df23["sexo"] = df23["sexo"].map(map_sexo)

# 3) Comprobar
print(df23["CCAA"].head(), df23["sexo"].head())
print(df23["CCAA"].isna().mean(), df23["sexo"].isna().mean())



0    País Vasco
1    País Vasco
2    País Vasco
3    País Vasco
4    País Vasco
Name: CCAA, dtype: object 0    Hombre
1     Mujer
2     Mujer
3     Mujer
4    Hombre
Name: sexo, dtype: object
0.0 0.0


In [ ]:
df23

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,año,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria,alcohol_md_sem,IMC,IMC_cat
0,País Vasco,Hombre,75,162,68,5.0,6.0,6.0,5.0,2023,0.5,0.0,0.0,0.5,1.0,Bajo,0,25.910684,Sobrepeso
1,País Vasco,Mujer,96,150,52,1.0,6.0,6.0,6.0,2023,7.0,0.0,0.0,0.0,7.0,Moderado,0,23.111111,Normopeso
2,País Vasco,Mujer,74,159,73,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto,0,28.875440,Sobrepeso
3,País Vasco,Mujer,79,162,60,1.0,6.0,6.0,5.0,2023,7.0,0.0,0.0,0.5,7.5,Alto,0,22.862369,Normopeso
4,País Vasco,Hombre,70,175,70,1.0,6.0,5.0,5.0,2023,7.0,0.0,0.5,0.5,8.0,Alto,4.29,22.857143,Normopeso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21027,Melilla,Mujer,33,165,69,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,0,25.344353,Sobrepeso
21028,Melilla,Mujer,31,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,2.86,22.491349,Normopeso
21029,Melilla,Hombre,39,170,65,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,8.57,22.491349,Normopeso
21030,Melilla,Hombre,42,180,74,4.0,4.0,4.0,4.0,2023,1.5,1.5,1.5,1.5,6.0,Moderado,5.71,22.839506,Normopeso


In [ ]:
df23["año"] = 2023


In [ ]:
cols_final = [
    "CCAA","sexo","edad","altura_cm","peso_kg",
    "freq_dulces","freq_refrescos","freq_fastfood","freq_snacks",
    "IMC_cat","alcohol_md_sem",
    "freq_dulces_sem","freq_refrescos_sem","freq_fastfood_sem","freq_snacks_sem",
    "ICU","ICU_categoria",
    "año"
]

df23 = df23.reindex(columns=cols_final)


In [ ]:
df23

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,IMC_cat,alcohol_md_sem,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria,año
0,País Vasco,Hombre,75,162,68,5.0,6.0,6.0,5.0,Sobrepeso,0,0.5,0.0,0.0,0.5,1.0,Bajo,2023
1,País Vasco,Mujer,96,150,52,1.0,6.0,6.0,6.0,Normopeso,0,7.0,0.0,0.0,0.0,7.0,Moderado,2023
2,País Vasco,Mujer,74,159,73,1.0,6.0,6.0,5.0,Sobrepeso,0,7.0,0.0,0.0,0.5,7.5,Alto,2023
3,País Vasco,Mujer,79,162,60,1.0,6.0,6.0,5.0,Normopeso,0,7.0,0.0,0.0,0.5,7.5,Alto,2023
4,País Vasco,Hombre,70,175,70,1.0,6.0,5.0,5.0,Normopeso,4.29,7.0,0.0,0.5,0.5,8.0,Alto,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21027,Melilla,Mujer,33,165,69,4.0,4.0,4.0,4.0,Sobrepeso,0,1.5,1.5,1.5,1.5,6.0,Moderado,2023
21028,Melilla,Mujer,31,170,65,4.0,4.0,4.0,4.0,Normopeso,2.86,1.5,1.5,1.5,1.5,6.0,Moderado,2023
21029,Melilla,Hombre,39,170,65,4.0,4.0,4.0,4.0,Normopeso,8.57,1.5,1.5,1.5,1.5,6.0,Moderado,2023
21030,Melilla,Hombre,42,180,74,4.0,4.0,4.0,4.0,Normopeso,5.71,1.5,1.5,1.5,1.5,6.0,Moderado,2023


In [ ]:
# Guardar ENSE 2023 limpio
df23.to_csv("Data_clean/ENSE_2023_clean.csv", index=False, encoding="utf-8")


In [ ]:
pd.read_csv("Data_clean/ENSE_2023_clean.csv").head()


,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,IMC_cat,alcohol_md_sem,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria,año
0,País Vasco,Hombre,75,162,68,5.0,6.0,6.0,5.0,Sobrepeso,0.00,0.5,0.0,0.0,0.5,1.0,Bajo,2023
1,País Vasco,Mujer,96,150,52,1.0,6.0,6.0,6.0,Normopeso,0.00,7.0,0.0,0.0,0.0,7.0,Moderado,2023
2,País Vasco,Mujer,74,159,73,1.0,6.0,6.0,5.0,Sobrepeso,0.00,7.0,0.0,0.0,0.5,7.5,Alto,2023
3,País Vasco,Mujer,79,162,60,1.0,6.0,6.0,5.0,Normopeso,0.00,7.0,0.0,0.0,0.5,7.5,Alto,2023
4,País Vasco,Hombre,70,175,70,1.0,6.0,5.0,5.0,Normopeso,4.29,7.0,0.0,0.5,0.5,8.0,Alto,2023


In [ ]:
df17 = pd.read_csv("Data_clean/ENSE_2017_clean.csv")
df20 = pd.read_csv("Data_clean/ENSE_2020_clean.csv")
df23 = pd.read_csv("Data_clean/ENSE_2023_clean.csv")

df_all = pd.concat([df17, df20, df23], ignore_index=True)


In [ ]:
df_all["año"] = pd.to_datetime(df_all["año"], format="%Y", errors="coerce")



In [ ]:
df_all["año"].dtype


dtype('<M8[ns]')

In [ ]:
df_all

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,IMC_cat,alcohol_md_sem,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria,año
0,País Vasco,Mujer,84.0,158.0,73.0,1.0,6.0,6.0,6.0,Sobrepeso,0.00,7.0,0.0,0.0,0.0,7.0,Moderado,2017-01-01
1,País Vasco,Hombre,26.0,182.0,65.0,3.0,1.0,1.0,4.0,Normopeso,0.00,3.0,7.0,7.0,1.5,18.5,Alto,2017-01-01
2,País Vasco,Mujer,56.0,160.0,50.0,1.0,6.0,6.0,4.0,Normopeso,0.00,7.0,0.0,0.0,1.5,8.5,Alto,2017-01-01
3,País Vasco,Hombre,70.0,164.0,64.0,1.0,5.0,5.0,4.0,Normopeso,44.29,7.0,0.5,0.5,1.5,9.5,Alto,2017-01-01
4,País Vasco,Hombre,61.0,167.0,80.0,5.0,5.0,5.0,5.0,Sobrepeso,45.71,0.5,0.5,0.5,0.5,2.0,Bajo,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66188,Melilla,Mujer,33.0,165.0,69.0,4.0,4.0,4.0,4.0,Sobrepeso,0.00,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01
66189,Melilla,Mujer,31.0,170.0,65.0,4.0,4.0,4.0,4.0,Normopeso,2.86,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01
66190,Melilla,Hombre,39.0,170.0,65.0,4.0,4.0,4.0,4.0,Normopeso,8.57,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01
66191,Melilla,Hombre,42.0,180.0,74.0,4.0,4.0,4.0,4.0,Normopeso,5.71,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01


In [ ]:
df_all

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,IMC_cat,alcohol_md_sem,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria,año
0,País Vasco,Mujer,84.0,158.0,73.0,1.0,6.0,6.0,6.0,Sobrepeso,0.00,7.0,0.0,0.0,0.0,7.0,Moderado,2017-01-01
1,País Vasco,Hombre,26.0,182.0,65.0,3.0,1.0,1.0,4.0,Normopeso,0.00,3.0,7.0,7.0,1.5,18.5,Alto,2017-01-01
2,País Vasco,Mujer,56.0,160.0,50.0,1.0,6.0,6.0,4.0,Normopeso,0.00,7.0,0.0,0.0,1.5,8.5,Alto,2017-01-01
3,País Vasco,Hombre,70.0,164.0,64.0,1.0,5.0,5.0,4.0,Normopeso,44.29,7.0,0.5,0.5,1.5,9.5,Alto,2017-01-01
4,País Vasco,Hombre,61.0,167.0,80.0,5.0,5.0,5.0,5.0,Sobrepeso,45.71,0.5,0.5,0.5,0.5,2.0,Bajo,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66188,Melilla,Mujer,33.0,165.0,69.0,4.0,4.0,4.0,4.0,Sobrepeso,0.00,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01
66189,Melilla,Mujer,31.0,170.0,65.0,4.0,4.0,4.0,4.0,Normopeso,2.86,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01
66190,Melilla,Hombre,39.0,170.0,65.0,4.0,4.0,4.0,4.0,Normopeso,8.57,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01
66191,Melilla,Hombre,42.0,180.0,74.0,4.0,4.0,4.0,4.0,Normopeso,5.71,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01


In [ ]:
df_all.to_csv("Data_clean/ENSE_All_clean.csv", index=False, encoding="utf-8")

In [ ]:
pd.read_csv("Data_clean/ENSE_All_clean.csv")

,CCAA,sexo,edad,altura_cm,peso_kg,freq_dulces,freq_refrescos,freq_fastfood,freq_snacks,IMC_cat,alcohol_md_sem,freq_dulces_sem,freq_refrescos_sem,freq_fastfood_sem,freq_snacks_sem,ICU,ICU_categoria,año
0,País Vasco,Mujer,84.0,158.0,73.0,1.0,6.0,6.0,6.0,Sobrepeso,0.00,7.0,0.0,0.0,0.0,7.0,Moderado,2017-01-01
1,País Vasco,Hombre,26.0,182.0,65.0,3.0,1.0,1.0,4.0,Normopeso,0.00,3.0,7.0,7.0,1.5,18.5,Alto,2017-01-01
2,País Vasco,Mujer,56.0,160.0,50.0,1.0,6.0,6.0,4.0,Normopeso,0.00,7.0,0.0,0.0,1.5,8.5,Alto,2017-01-01
3,País Vasco,Hombre,70.0,164.0,64.0,1.0,5.0,5.0,4.0,Normopeso,44.29,7.0,0.5,0.5,1.5,9.5,Alto,2017-01-01
4,País Vasco,Hombre,61.0,167.0,80.0,5.0,5.0,5.0,5.0,Sobrepeso,45.71,0.5,0.5,0.5,0.5,2.0,Bajo,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66188,Melilla,Mujer,33.0,165.0,69.0,4.0,4.0,4.0,4.0,Sobrepeso,0.00,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01
66189,Melilla,Mujer,31.0,170.0,65.0,4.0,4.0,4.0,4.0,Normopeso,2.86,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01
66190,Melilla,Hombre,39.0,170.0,65.0,4.0,4.0,4.0,4.0,Normopeso,8.57,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01
66191,Melilla,Hombre,42.0,180.0,74.0,4.0,4.0,4.0,4.0,Normopeso,5.71,1.5,1.5,1.5,1.5,6.0,Moderado,2023-01-01


In [ ]:
from pathlib import Path
Path.cwd()



WindowsPath('c:/Users/nuez_/Desktop/Proyecto ENSE')

In [ ]:
from pathlib import Path
import pandas as pd

BASE = Path.cwd() / "Data_clean"
print("Buscando en:", BASE)
print("Existe Data_clean?", BASE.exists())

df = pd.read_csv(BASE / "ENSE_All_clean.csv")
print("Filas:", len(df), "| CCAA distintas:", df["CCAA"].nunique())


Buscando en: c:\Users\nuez_\Desktop\Proyecto ENSE\Data_clean
Existe Data_clean? True
Filas: 66193 | CCAA distintas: 27


In [ ]:
vc = df["CCAA"].value_counts(dropna=False)
print("Nº valores distintos:", df["CCAA"].nunique(dropna=False))
display(vc.head(40))


Nº valores distintos: 27


CCAA
Andalucía                      8429
Cataluña                       6303
Madrid, Comunidad de           4359
País Vasco                     4087
Castilla y León                3755
Comunitat Valenciana           3701
Galicia                        3543
Canarias                       3132
Extremadura                    2974
Aragón                         2910
Cantabria                      2361
Castilla - La Mancha           2273
C. Valenciana                  2059
Murcia, Región de              2033
Madrid                         1876
Asturias, Principado de        1818
Navarra, Comunidad Foral de    1524
Balears, Illes                 1319
Rioja, La                      1318
Región de Murcia               1080
Castilla-La Mancha             1030
Navarra                         854
Melilla                         801
Asturias                        705
Ceuta                           690
Illes Balears                   680
La Rioja                        579
Name: count, dtype: int

In [ ]:
ccaa_vals = sorted(df["CCAA"].dropna().unique())
ccaa_vals


['Andalucía',
 'Aragón',
 'Asturias',
 'Asturias, Principado de',
 'Balears, Illes',
 'C. Valenciana',
 'Canarias',
 'Cantabria',
 'Castilla - La Mancha',
 'Castilla y León',
 'Castilla-La Mancha',
 'Cataluña',
 'Ceuta',
 'Comunitat Valenciana',
 'Extremadura',
 'Galicia',
 'Illes Balears',
 'La Rioja',
 'Madrid',
 'Madrid, Comunidad de',
 'Melilla',
 'Murcia, Región de',
 'Navarra',
 'Navarra, Comunidad Foral de',
 'País Vasco',
 'Región de Murcia',
 'Rioja, La']